# 大作業
[MRI腦瘤判斷](https://www.kaggle.com/c/izcc-ailecture-1)

## Step1:準備資料
Tudo這裡好心，已經先準備好了

**建議! 把檔案放進drive裡面**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/izcc-ailecture-1.zip
!mkdir train_data
!mv /content/train_Brain_Tumor/train_Brain_Tumor train_data
!mv /content/train_Healthy/train_Healthy train_data


unzip:  cannot find or open /content/izcc-ailecture-1.zip, /content/izcc-ailecture-1.zip.zip or /content/izcc-ailecture-1.zip.ZIP.
mv: cannot stat '/content/train_Brain_Tumor/*': No such file or directory
mv: cannot stat '/content/train_Healthy/*': No such file or directory


In [ ]:
import glob
ffs = glob.glob('/content/train_data/*/*')
import imageio
data = []
for f in ffs:
  if 'Brain_Tumor' in f:
    data.append([f, 1])
  else:
    data.append([f, 0])

In [ ]:
transform = A.Compose([
    A.Resize(224, 224),
])

In [ ]:
# 讀取和標記數據
ffs = glob.glob('train_data/*/*')
data = []
all_list = []
for f in tqdm.tqdm(ffs):
    im = imageio.imread(f)
    im = Image.fromarray(im).convert('L')
    im = np.array(im)


    if 'Brain_Tumor' in f:
        data.append([im, 1])
        all_list.append(1)
    else:
        data.append([im, 0])
        all_list.append(0)

np.bincount(all_list)

In [ ]:
import numpy as np
import torch

def preprocess_image(image, transform, label, device):
    # 將圖像進行預處理
    im = transform(image=image)['image']
    # 將圖像擴展為 3 通道
    im = np.stack([im] * 3, axis=0)[None, ...]  # 1, 3, 224, 224
    # 將圖像正規化
    im = im / np.max(im)
    # 將 NumPy 陣列轉換為 PyTorch 張量
    im_d = torch.from_numpy(im).to(device).float()
    labels = torch.from_numpy(np.array([label])).to(device)

    return im_d, labels


In [ ]:
!pip install onnx
def transfer_to_onnx(model,name):
  model.eval()
  example_input = torch.randn(1, 3, 224, 224)
  output_onnx_path = f"{name}.onnx"
  torch.onnx.export(model, example_input, output_onnx_path, verbose=True)